In [5]:
from langchain.embeddings import OpenAIEmbeddings

from langchain.vectorstores import Chroma

from langchain.chains import RetrievalQA, ConversationalRetrievalChain

from langchain.memory import ConversationBufferMemory

from langchain.llms import OpenAI

from langchain.chat_models import ChatOpenAI


from dotenv import load_dotenv

import os

import openai

# Azure
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_EMBEDDING_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")
OPENAI_EMBEDDING_VERSION = os.getenv("OPENAI_EMBEDDING_VERSION")

#init Azure OpenAI
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

In [10]:
from langchain.chat_models import AzureChatOpenAI
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', chunk_size=1)
db = Chroma(persist_directory='./vectordb', embedding_function=embeddings)
retriever = db.as_retriever(search_kwargs={"k": 3})
llm = AzureChatOpenAI(deployment_name=OPENAI_DEPLOYMENT_NAME,
                      model_name=OPENAI_MODEL_NAME,
                      openai_api_base=OPENAI_DEPLOYMENT_ENDPOINT,
                      openai_api_version=OPENAI_EMBEDDING_VERSION,
                      openai_api_key=OPENAI_API_KEY)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
bitcoin = ConversationalRetrievalChain.from_llm(llm, retriever, memory=memory)
result = bitcoin({
    "question": "best hotels"
})
result

{'question': 'best hotels',
 'chat_history': [HumanMessage(content='best hotels', additional_kwargs={}, example=False),
  AIMessage(content='The article mentions several hotels, some of which are:\n\n- Majorda Beach Resort: a starred hotel located in Majorda.\n- 5-star hotel in Bogmalo: located right on the edge of Bogmalo beach.\n- Several air-conditioned resort complexes, tourist cottages, and guest houses in Colva beach that offer modern amenities.\n- There are also several smaller and appealing places to stay in Bogmalo.', additional_kwargs={}, example=False)],
 'answer': 'The article mentions several hotels, some of which are:\n\n- Majorda Beach Resort: a starred hotel located in Majorda.\n- 5-star hotel in Bogmalo: located right on the edge of Bogmalo beach.\n- Several air-conditioned resort complexes, tourist cottages, and guest houses in Colva beach that offer modern amenities.\n- There are also several smaller and appealing places to stay in Bogmalo.'}